# SQL Data Aggregation

This file initiates a new database called "cairo.db", then iterate through all .csv files to convert them into pandas dataframes, and then to tables in the database with the same names of the files. Finally it shows some basic stats and city insights.

In [1]:
import sqlite3
import pandas as pd
from IPython.display import display
from sqlalchemy import create_engine

### Creating the SQLite Database:

In [2]:
cairo_db = create_engine('sqlite:///cairo.db')

def create_table(filename, tablename): 
    for df in pd.read_csv(filename, encoding = 'utf-8', iterator = True):
        df.to_sql(tablename, cairo_db, index = False, if_exists = 'replace')

In [3]:
# iterate through the .csv files to convert them into pandas dataframes, and then to tables in the SQLite database 
create_table('nodes.csv', 'nodes')
create_table('nodes_tags.csv', 'nodes_tags')
create_table('ways.csv', 'ways')
create_table('ways_nodes.csv', 'ways_nodes')
create_table('ways_tags.csv', 'ways_tags')

In [4]:
# opening a connection with the database
db = sqlite3.connect('cairo.db')
cursor = db.cursor()

### Data Exploration:

Displaying the first 5 rows from each table for exploration

In [5]:
def show_head(query, table_name):
    df = pd.read_sql_query(query, db)
    print table_name
    display(df.head())

In [6]:
# show first 5 rows from nodes table
query = 'SELECT * FROM nodes'
show_head(query, 'nodes')

nodes


,id,lat,lon,version
0,27085092,29.980120,31.135403,1
1,27085093,29.978125,31.135394,1
2,27085094,29.978133,31.133074,1
3,27085096,29.976907,31.131866,1
4,27085097,29.975016,31.131884,1


In [7]:
# show first 5 rows from nodes_tags table
query = 'SELECT * FROM nodes_tags'
show_head(query, 'nodes_tags')

nodes_tags


,id,key,value,type
0,27565120,name,الجيزة,regular
1,27565120,note,reverted to arabic,regular
2,27565120,is_in,"Giza, Egypt",regular
3,27565120,place,city,regular
4,27565120,ar,الجيزة,name


In [8]:
# show first 5 rows from ways table
query = 'SELECT * FROM ways'
show_head(query, 'ways')

ways


,id,version
0,4420396,1
1,4420397,1
2,4420431,1
3,4943478,1
4,4943479,1


In [9]:
# show first 5 rows from ways_tags table
query = 'SELECT * FROM ways_tags'
show_head(query, 'ways_tags')

ways_tags


,id,key,value,type
0,4420396,fee,yes,regular
1,4420396,name,هرم خفرع,regular
2,4420396,tomb,pyramid,regular
3,4420396,charge,1.1$,regular
4,4420396,height,136,regular


In [10]:
# show first 5 rows from ways_nodes table
query = 'SELECT * FROM ways_nodes'
show_head(query, 'ways_nodes')

ways_nodes


,id,node_id,position
0,4420396,27085096,0
1,4420396,27085097,1
2,4420397,27085092,0
3,4420397,27085093,1
4,4420397,27085094,2


### Cairo Insights:

Some interesting insights generated by SQL queries..

In [11]:
# total no. of nodes
query = 'SELECT COUNT(*) FROM nodes'
cursor.execute(query)
'Number of nodes: {}'.format(cursor.fetchall()[0][0])

'Number of nodes: 217818'

In [12]:
# total no. of ways
query = 'SELECT COUNT(*) FROM ways'
cursor.execute(query)
'Number of ways: {}'.format(cursor.fetchall()[0][0])

'Number of ways: 37628'

In [13]:
def show_result(query, rows, description):
    df = pd.read_sql_query(query, db)
    print description
    display(df.head(rows))

In [14]:
# top 5 used languages in Cairo
query = 'SELECT key, COUNT(*) FROM nodes_tags WHERE type = "name" GROUP BY 1 ORDER BY COUNT(*) DESC LIMIT 5'
show_result(query, 5, 'Top 5 used languages in Cairo')

Top 5 used languages in Cairo


,key,COUNT(*)
0,en,3302
1,ar,2782
2,de,238
3,fr,226
4,ar1,148


In [15]:
# most common shop types
query = 'SELECT value, COUNT(*) FROM nodes_tags WHERE key = "shop" GROUP BY 1 ORDER BY COUNT(*) DESC LIMIT 10'
show_result(query, 10, 'Most common shop types')

Most common shop types


,value,COUNT(*)
0,supermarket,111
1,clothes,51
2,convenience,47
3,bakery,42
4,kiosk,32
5,dry_cleaning,26
6,books,25
7,car,24
8,confectionery,22
9,mobile_phone,21


In [16]:
# top 10 banks 
query = 'WITH banks AS (SELECT DISTINCT id AS bank_node_id FROM nodes_tags WHERE value = "bank") SELECT value, COUNT(*) FROM nodes_tags INNER JOIN banks ON banks.bank_node_id = nodes_tags.id WHERE key = "en" AND type = "name" GROUP BY 1 ORDER BY COUNT(*) DESC LIMIT 10'
show_result(query, 10, 'Top 10 banks')

Top 10 banks


,value,COUNT(*)
0,CIB,10
1,NSGB,10
2,Al-Bank Al-Ahly,6
3,BNP Parisbas,4
4,Bank Misr,4
5,Credit Agricole Egypt,4
6,Crédit Agricole,4
7,Faisal Islamic Bank,4
8,The National Bank of Egypt,4
9,ABC,2


In [17]:
# closing the connection with the database
cursor.close()
db.close()